### Introduction
- Netmarble 사의 Fate/grandorder googleplaystore review crawling
- 작동 방식은 브라운더스트 crawling과 동일

#### Import libraries

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
driver = webdriver.Chrome('./chromedriver')
review_url = 'https://play.google.com/store/apps/details?id=com.netmarble.fgok&showAllReviews=true'
driver.get(review_url) #접속할 Review

#### Web page control

In [3]:
time_now = time.time()
flag=0 
while 1:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        loadMore=driver.find_element_by_xpath("//*[contains(@class,'U26fgb O0WRkf oG5Srb C0oVfc n9lfJ')]").click()
        time.sleep(1)
    except:
        time.sleep(1)
        flag=flag+1
        if flag >= 100: 
            break
    else:
        flag=0
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
print("{} seconds spent to viewing all reviews".format(time.time()-time_now))

9019.22232580185 seconds spent to viewing all reviews


In [4]:
table_dict = {}
date_list = []
contents_list = []
stars_list = []

#reviews=driver.find_elements_by_xpath("//*div[@class='d15Mdf bAhLNe']")
reviews=driver.find_elements_by_xpath("//*[@id='fcxH9b']//div[@class='d15Mdf bAhLNe']")

In [5]:
print(len(reviews))

13634


#### Rating

In [6]:
star_cnt = 0
for review in reviews: #text로 설명된 rating
    soup=BeautifulSoup(review.get_attribute("innerHTML"),"lxml")
    ratings=soup.find('div',role='img').get('aria-label')
    stars_list.append(ratings)
    star_cnt += 1

print("{}개의 review를 수집합니다.".format(star_cnt))

13634개의 review를 수집합니다.


#### Review text

In [7]:
for i in range(1,star_cnt+1): #review text
    review_txt = driver.find_elements_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div/div['+ str(i) + ']/div/div[2]/div[2]/span[1]')[0].text
    contents_list.append(review_txt)
    if i%1000==0:
        print("{}th review text".format(i))

1000th review text
2000th review text
3000th review text
4000th review text
5000th review text
6000th review text
7000th review text
8000th review text
9000th review text
10000th review text
11000th review text
12000th review text
13000th review text


#### Review date

In [8]:
for i in range(1,star_cnt+1): #review date
    review_date = driver.find_elements_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div/div['+ str(i) + ']/div/div[2]/div[1]/div[1]/div/span[2]')[0].text
    date_list.append(review_date)
    if i%1000==0:
        print("{}th review date".format(i))

1000th review date
2000th review date
3000th review date
4000th review date
5000th review date
6000th review date
7000th review date
8000th review date
9000th review date
10000th review date
11000th review date
12000th review date
13000th review date


In [9]:
print(len(date_list), len(contents_list), len(stars_list))

13634 13634 13634


#### Saving Data Frame 

In [10]:
table_dict['time'] = date_list
table_dict['review_txt'] = contents_list
table_dict['stars_txt'] = stars_list

review_df = pd.DataFrame.from_dict(table_dict)
review_df['time'] = review_df.loc[:,'time'].str.replace(" ","")
review_df['time'] = pd.to_datetime(review_df.loc[:,'time'], format = '%Y년%m월%d일')
print('가장 최근 작성 review 날짜 : {} '.format(review_df['time'].max()))
print('가장 오래전 작성 review 날짜 : {} '.format(review_df['time'].min()))

review_df['ratings'] = review_df['stars_txt'].str[10]
review_df['ratings'] = review_df['ratings'].astype(int)

print('review 별 평점 분포', '\n', review_df['ratings'].value_counts())
print('review 별 평점 분포 %', '\n', review_df['ratings'].value_counts(normalize = True))
    
review_df.to_csv('review_crwl_FateGrand.csv', index = False)

driver.close()

가장 최근 작성 review 날짜 : 2019-05-24 00:00:00 
가장 오래전 작성 review 날짜 : 2017-11-21 00:00:00 
review 별 평점 분포 
 5    8431
4    2235
1    1510
3    1083
2     375
Name: ratings, dtype: int64
review 별 평점 분포 % 
 5    0.618381
4    0.163928
1    0.110753
3    0.079434
2    0.027505
Name: ratings, dtype: float64
